In [1]:
import numpy as np
from data import get_data
from model import *
from model_constraints import *

In [2]:
def get_variables(data):
    employee_dv =  {(m, d, t): 0
                for m in data["STAFF"]
                for d in data["DAYS"]
                for t in data["SHIFTS"]}

    above_dv = {(d, t, k): 0
                for d in data["DAYS"]
                for t in data["SHIFTS"]
                for k in data["SCENARIOS"]}

    below_dv = {(d, t, k): 0
                for d in data["DAYS"]
                for t in data["SHIFTS"]
                for k in data["SCENARIOS"]}
    return employee_dv, above_dv, below_dv

def get_results(data, file, employee_dv, above_dv, below_dv):
    with open(file) as soltext:
        count = [0]  * len(data["DAYS"])
        soltext.readline()
        soltext.readline()
        soltext.readline()
        for d in data["DAYS"]:
            assert(soltext.readline() == f"Day: {d}\n")

            while True:
                line = soltext.readline()
                if line == '\n':
                    break
                else:
                    m, t = line.strip().split(" works shift ")
                    employee_dv[m,d,t] = 1
                    count[d - 1] += 1

    for d in data["DAYS"]:
        for t in data["SHIFTS"]:
            for k in data["SCENARIOS"]:
                requirement = [item for item in data[f"SECTION_COVER{k}"] if item[0] == d - 1 and item[1] == t][0]
                coverage_req = requirement[2]
                if count[d - 1] > coverage_req:
                    above_dv[d, t, k] = count[d - 1] - coverage_req
                elif count[d - 1] < coverage_req:
                    below_dv[d, t, k] = coverage_req - count[d - 1]

In [7]:
for index in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, "fa"]:
    instance = 5
    np.random.seed(instance)
    data = get_data(f"instance{instance}.txt")

    a, b, c = get_variables(data)
    get_results(data, f"output/instance{instance}s_{index}.txt", a, b, c)

    for idx in range(0, 10):
        model = create_model()

        employee_dv = create_employee_dv(model, "assigned", data["STAFF"], data["DAYS"], data["SHIFTS"])
        above_dv = create_above_coverage_dv(model, len(data["STAFF"]), "above", data["DAYS"], data["SHIFTS"], [idx])
        below_dv = create_below_coverage_dv(model, len(data["STAFF"]), "below", data["DAYS"], data["SHIFTS"], [idx])
        
        for d in data["DAYS"]:
            for t in data["SHIFTS"]:
                for s in data["STAFF"]:
                    model.Add(employee_dv[s, d, t] == a[s, d, t])

        weekend_dv = create_weekend_dv(model, "weekend", data["STAFF"], data["WEEKENDS"])
        add_dv = create_add_dv(model, len(data["STAFF"]) // 2, "above", data["DAYS"], data["SHIFTS"], [idx])
        subtract_dv = create_subtract_dv(model, len(data["STAFF"]) // 2, "below", data["DAYS"], data["SHIFTS"], [idx])

        one_shift_per_day(model, data, employee_dv)
        shift_rotation(model, data, employee_dv)
        max_shifts(model, data, employee_dv)

        max_work_time(model, data, employee_dv)
        min_work_time(model, data, employee_dv)

        max_consecutive_shifts(model, data, employee_dv)
        min_consecutive_shifts(model, data, employee_dv)
        min_consecutive_shifts_off(model, data, employee_dv)

        max_weekends(model, data, employee_dv, weekend_dv)

        days_off(model, data, employee_dv)

        coverage2(model, data, employee_dv, above_dv, below_dv, add_dv, subtract_dv, [idx])

        # All constraints together takes 1.5 hours to find a feasible solution
        import time

        start = time.time()

        model.Minimize(
                #off requests not applied
                sum(data["SECTION_SHIFT_OFF_REQUESTS"].get((s, d - 1, t), 0) * employee_dv[(s, d, t)]
                    for s in data["STAFF"]
                    for d in data["DAYS"]
                    for t in data["SHIFTS"])
                +
                #on requests not applied
                sum(data["SECTION_SHIFT_ON_REQUESTS"].get((s, d - 1, t), 0) * (1 - employee_dv[(s, d, t)])
                    for s in data["STAFF"]
                    for d in data["DAYS"]
                    for t in data["SHIFTS"])
                +
                sum([
                #overstaffing penalty
                (1 / len([idx])) * sum(above_dv[d, t, k] * [item[4] for item in data[f"SECTION_COVER{k}"] if item[0] == d - 1 and item[1] == t][0]
                    for d in data["DAYS"]
                    for t in data["SHIFTS"]
                    for k in [idx])
                + 
                #understaffing penalty
                (1 / len([idx])) * sum(below_dv[d, t, k] * [item[3] for item in data[f"SECTION_COVER{k}"] if item[0] == d - 1 and item[1] == t][0]
                    for d in data["DAYS"]
                    for t in data["SHIFTS"]
                    for k in [idx])
                +
                (1 / len([idx])) * sum(add_dv[d, t, k] * 100
                    for d in data["DAYS"]
                    for t in data["SHIFTS"]
                    for k in [idx])
                +
                (1 / len([idx])) * sum(subtract_dv[d, t, k] * 40
                    for d in data["DAYS"]
                    for t in data["SHIFTS"]
                    for k in [idx])
                +
                #readd
                (1 / len(data["SCENARIOS"])) * sum(add_dv[d, t, k] * ([item[3] for item in data[f"SECTION_COVER{k}"] if item[0] == d - 1 and item[1] == t][0] * 0.75)
                    for d in data["DAYS"]
                    for t in data["SHIFTS"]
                    for k in [idx])
                + 
                #remove
                (1 / len(data["SCENARIOS"])) * sum(subtract_dv[d, t, k] * ([item[4] for item in data[f"SECTION_COVER{k}"] if item[0] == d - 1 and item[1] == t][0] * 0.75)
                    for d in data["DAYS"]
                    for t in data["SHIFTS"]
                    for k in [idx])
                ]
                )
        )

        solver = create_solver(1200)
        status = solver.Solve(model)

        print(solver.StatusName(status))
        import collections
        results = collections.defaultdict(list)
        for d in data["DAYS"]:
            results[d] = []

        with open(f"output/instance{instance}_{index}/instance{instance}s_{idx}fa.txt", 'w') as file:
            file.write(f"{status}\n")
            file.write('Maximum of objective function: %i' % solver.ObjectiveValue() + '\n\n')
            print()
            for d in data["DAYS"]:
                file.write(f'Day: {d}\n')
                for s in data["SHIFTS"]:
                    for m in data["STAFF"]:
                        if solver.Value(employee_dv[(m, d, s)]) == 1:
                            results[d].append(m)
                            file.write(f"{m} works shift {s}\n")
                file.write("\n")

                end = time.time()
            print(f"time elapsed: {end - start}")

OPTIMAL

time elapsed: 0.05358314514160156
OPTIMAL

time elapsed: 0.0485377311706543
OPTIMAL

time elapsed: 0.03855323791503906
OPTIMAL

time elapsed: 0.07955336570739746
OPTIMAL

time elapsed: 0.04656791687011719
OPTIMAL

time elapsed: 0.04853487014770508
OPTIMAL

time elapsed: 0.04953813552856445
OPTIMAL

time elapsed: 0.05599856376647949
OPTIMAL

time elapsed: 0.05215907096862793
OPTIMAL

time elapsed: 0.04781317710876465
OPTIMAL

time elapsed: 0.04173898696899414
OPTIMAL

time elapsed: 0.04761099815368652
OPTIMAL

time elapsed: 0.046005964279174805
OPTIMAL

time elapsed: 0.04001212120056152
OPTIMAL

time elapsed: 0.04100465774536133
OPTIMAL

time elapsed: 0.04824185371398926
OPTIMAL

time elapsed: 0.052036285400390625
OPTIMAL

time elapsed: 0.05402493476867676
OPTIMAL

time elapsed: 0.048119306564331055
OPTIMAL

time elapsed: 0.04493117332458496
OPTIMAL

time elapsed: 0.05300450325012207
OPTIMAL

time elapsed: 0.06799554824829102
OPTIMAL

time elapsed: 0.04809427261352539
OPTIMAL

